# Review MRI

In [ ]:
# import visualization functions
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

sys.path.append("/data/gpfs/projects/punim1836/CT-MRI_LandmarkDetection/src/models")
sys.path

In [ ]:
import imageio as iio
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt

dicom_AH_MR_path = '/data/gpfs/projects/punim1836/Data/raw/CT_MRI_Pre_Post/GP MR'
head_vol = iio.volread(dicom_AH_MR_path, 'DICOM')

head_vol.shape

# [[[539.  410.5  44. ]
#   [534.5 426.   42. ]
#   [241.  421.   50. ]
#   [248.  436.   47. ]]

plt.imshow(head_vol[44], cmap='grey')
plt.axis('off')
plt.show()

In [ ]:
sampling = head_vol.meta['sampling']
pixel_space = [sampling[1], sampling[2], sampling[0]]
print(pixel_space)

In [ ]:
head_vol_array = np.moveaxis(np.asarray(head_vol), 0 ,2)

In [ ]:
base_save_dir = "../data/processed/Y"

roi_MR_medium = np.load(f"{base_save_dir}/ROI_MR_Medium.npy")

In [ ]:
import numpy as np
import importlib
import common.MyDataset as MyDataset
import common.Visualization as Visualization

importlib.reload(Visualization)

print("roi: ", roi_MR_medium[0])

Visualization.show_pts(head_vol_array, roi_MR_medium[0], pixel_space)

## original MRI

In [ ]:
import h5py
import numpy as np

file_path = "/data/gpfs/projects/punim1836/Data/raw/aug/original_divided_augmentation"
file = h5py.File(f"{file_path}/SM_aug_1.mat", 'r')

load_mat_vol = file.get("augVol")
load_mat_pts = file.get("augPts")

volume = np.array(load_mat_vol).T
pts = np.array(load_mat_pts).reshape(3, 4).T

In [ ]:
import importlib
import common.MyDataset as MyDataset
import common.Visualization as Visualization

importlib.reload(Visualization)

pixel_space = [0.2604, 0.2604, 0.2604]
print("roi: ", pts)

Visualization.show_pts(volume, pts, pixel_space)

## augmented MRI

In [ ]:
import h5py
import numpy as np

file_path = "/data/gpfs/projects/punim1836/Data/raw/aug/reduce_size/176x88x48"
file = h5py.File(f"{file_path}/SM_176x88x48_3.mat", 'r')

load_mat_vol_left = file.get("augLeftVolRescaled")
load_mat_pts_left = file.get("augLeftPtsRescaled")
load_mat_res_left = file.get("leftRes")

volume_left = np.array(load_mat_vol_left).T
pts_left = np.array(load_mat_pts_left).reshape(3, 2).T
res_left = np.array(load_mat_res_left).T

load_mat_vol_right = file.get("augRightVolRescaled")
load_mat_pts_right = file.get("augRightPtsRescaled")
load_mat_res_right = file.get("rightRes")

volume_right = np.array(load_mat_vol_right).T
pts_right = np.array(load_mat_pts_right).reshape(3, 2).T
res_right = np.array(load_mat_res_right).T

In [ ]:
print(pts_left)
print(pts_right)

In [ ]:
import importlib
import common.MyDataset as MyDataset
import common.Visualization as Visualization

importlib.reload(Visualization)

pixel_space = res_left[0]

Visualization.show_two_landmarks(volume_left, pts_left, volume_right, pts_right, pixel_space)

In [ ]:
print(res_left.shape)
print(res_left)

# Pre-Process MRI

## rescale and divide? (Down)

Just used the original augmentation code of CT.

# Prepare Registration

## combine divided images (CT and MRI)

In [16]:
import numpy as np
# diff of Ground Truth and Prediction (Test dataset)
base_dir = "/data/gpfs/projects/punim1836/CT-MRI_LandmarkDetection/models/cropped/63x50x61/noises_s1_test_dis/two_landmarks/straight_model/turner-results/training-process/09Jul2024-12:12:48-trainID-33.3"
Y_test_true = np.load(f"{base_dir}/Y_test_true.npy")
Y_test_pred = np.load(f"{base_dir}/best_val_Y_test_pred.npy")

diff_CT_cropped = Y_test_true - Y_test_pred

# voxel distance, resolution is [0.2604, 0.2604, 0.2604]
diff_CT = np.copy(diff_cropped)
for idx in range(200):
    diff_CT[2*idx+1, :, 0] = - diff_CT[2*idx+1, :, 0]

print(diff_CT_cropped[0:2])
print("***")
print(diff_CT[0:2])

[[[-0.17109299  1.159277   -0.17753983]
  [ 1.3277473  -2.6857376   0.27730942]]

 [[-0.16573906  1.7073612  -1.0718613 ]
  [ 0.6180935  -1.1416245   0.5216465 ]]]
***
[[[-0.00392151 -0.43639755  0.31114578]
  [-0.32333374 -1.9399071   0.55685043]]

 [[-0.582592   -0.13200569  0.61296844]
  [ 0.541008   -2.949215    1.6897984 ]]]


In [17]:
import numpy as np
# diff of Ground Truth and Prediction (Test dataset)
base_dir = "/data/gpfs/projects/punim1836/CT-MRI_LandmarkDetection/models_MR/cropped_MR/72x72x72/noises_s1_test_dis/two_landmarks/straight_model/learning_rate/0.0001/16Sep2024-11:46:21-trainID-7.1"
Y_test_true = np.load(f"{base_dir}/Y_test_true.npy")
Y_test_pred = np.load(f"{base_dir}/best_val_Y_test_pred.npy")

diff_MR_cropped = Y_test_true - Y_test_pred

# voxel distance, resolution is [0.2604, 0.2604, 0.2604]
diff_MR = np.copy(diff_cropped)
for idx in range(200):
    diff_MR[2*idx+1, :, 0] = - diff_MR[2*idx+1, :, 0]

print(diff_MR_cropped[0:2])
print("***")
print(diff_MR[0:2])

[[[-0.00392151 -0.43639755  0.31114578]
  [-0.32333374 -1.9399071   0.55685043]]

 [[ 0.582592   -0.13200569  0.61296844]
  [-0.541008   -2.949215    1.6897984 ]]]
***
[[[-0.00392151 -0.43639755  0.31114578]
  [-0.32333374 -1.9399071   0.55685043]]

 [[-0.582592   -0.13200569  0.61296844]
  [ 0.541008   -2.949215    1.6897984 ]]]


In [18]:
from scipy.io import savemat

mdic_MR = {"diff_MR": diff_MR}
savemat("/data/gpfs/projects/punim1836/CT-MRI_Registration/src/matlab/for_Shawn/Data/MR/org/diff_MR.mat", mdic_MR)

mdic_CT = {"diff_CT": diff_CT}
savemat("/data/gpfs/projects/punim1836/CT-MRI_Registration/src/matlab/for_Shawn/Data/CT/org/diff_CT.mat", mdic_CT)